In [1]:
import json
import os
from library_dicom.dicom_processor.model.Series import Series 
from library_dicom.dicom_processor.model.SeriesPT import SeriesPT
from library_dicom.dicom_processor.model.SeriesCT import SeriesCT
from library_dicom.dicom_processor.model.csv_reader.MaskBuilder import MaskBuilder

from library_dicom.dicom_processor.tools.folders import *

import numpy as np
import SimpleITK as sitk

In [2]:
#LIST FROM PARSE_FILTERED_DICOM_DATASET

In [2]:
nifti_directory = '/media/salim/DD 2To/AHL2011_NIFTI'

csv_directory = '/media/salim/DD 2To/AHL2011_CSV_DATA/AHL2011-CSV'

In [4]:
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_list_path_csv.json'

In [8]:
data = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        for path in reader[info] :
            data.append(path)

In [9]:
dataset = []
for i in range(0,len(data),5):
    subliste = []
    subliste.append(data[i])
    subliste.append(data[i+1])
    subliste.append(data[i+2])
    subliste.append(data[i+3])
    subliste.append(data[i+4])
    dataset.append(subliste)

In [10]:
print(len(dataset))


617


In [ ]:
#GENERE LA LISTE DES JSON AVEC UN UNCONSTANT SPACING, SAVE IT AS JSON 

In [ ]:

from library_dicom.dicom_processor.tools.folders import *
liste_unconstant_z_spacing = []
for liste in dataset : 
    print(dataset.index(liste))
    for i in range(0,4,2) : 
        serie_objet = Series(liste[i])
        serie_objet.get_numpy_array()
        if serie_objet.get_z_spacing() == 'Unconstant Spacing' : 
            print(liste)
            liste_unconstant_z_spacing.append(liste)
            list_z_spacing = serie_objet.calculate_z_spacing()
            print(list_z_spacing)
            #liste_unconstant_z_spacing.append(list_z_spacing)


#save it as json

write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_problem_unconstant_spacing', liste_unconstant_z_spacing)

In [ ]:
from library_dicom.dicom_processor.tools.folders import *
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_problem_unconstant_spacing', liste_unconstant_z_spacing)

In [ ]:
#On enleve les series avec un unconstant spacing 
for serie in liste_unconstant_z_spacing : 
    dataset.remove(serie)

In [ ]:
#SAVE AS JSON THE FINAL LIST WITHOUT UNCONSTANT SPACING ERROR

In [ ]:
from library_dicom.dicom_processor.tools.folders import *
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_final_list_dataset', dataset)


In [ ]:
print(len(dataset)) #589


In [30]:
#RECHARGEZ LE JSON AVEC LES SERIES 
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_final_list_dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

589


In [ ]:
#IF MASK IS CORRECT => GENERATE NIFTI 
#IF PHILIPS, OR MASK FALSE => PUT SERIE IN A NEW LISTE 

In [ ]:
untreated_liste = []
#CT et PT pas dans le même ordre suivant les séries 
#voir pour les ranger avant PT en premier puis CT 
#On traite meme les philips
#On met les erreurs de coté
#si erreur sur les ellipse on tolere
#si erreur polygone alors faut checker



#error = []


for serie in dataset : 
    try : 
        print(serie)
        print(dataset.index(serie))
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0])
            manufacturer = serie_pt_objet.get_series_details()['series']['Manufacturer'] 
            if 'philips' in manufacturer.lower() :
                untreated_liste.append(serie)
            else : 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
                size_matrix = serie_pt_objet.get_size_matrix()
                mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
                mask_4D = mask_objet.mask_array
                if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                    print("MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                    #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[2])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                    print("EXPORT NIFTI MASK")
                else : 
                    untreated_liste.append(serie)
        else : 
            serie_pt_objet = SeriesPT(serie[2])
            manufacturer = serie_pt_objet.get_series_details()['series']['Manufacturer'] 
            if 'philips' in manufacturer.lower() :
                untreated_liste.append(serie)
            else : 
                nifti_array = serie_pt_objet.get_numpy_array()
                study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
                size_matrix = serie_pt_objet.get_size_matrix()
                mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
                mask_4D = mask_objet.mask_array
                if mask_objet.is_correct_suv(nifti_array) == True : #si mask correct 
                    print("MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                     #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[0])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                    print("EXPORT NIFTI MASK")
                else : 
                    untreated_liste.append(serie)
                   


    except Exception as err : 
        print(err)
        #error.append(serie)

        


In [ ]:
len(untreated_liste) #392 

In [ ]:
#SAVE THE SERIE WITH FALSE MASK 
from library_dicom.dicom_processor.tools.folders import *
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_serie_with_false_mask', untreated_liste)
print(len(untreated_liste))

In [ ]:
#IMPORT SERIES WITH FALSE MASK,
#CHECK FLIP AXE Z + SMALL ERROR ON ELLIPSE 

In [13]:
#importer la liste untreated_serie pour la suite : 
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_serie_with_false_mask.json'
untreated_liste = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        untreated_liste.append(info)

print(len(untreated_liste)) #392

392


In [14]:
#SEPARE LES SERIES PHILIPS DES AUTRES SERIES UNTREATED
philips_serie = []
untreated_liste_2 = []
for serie in untreated_liste : 
    #print(untreated_liste.index(serie))
    try : 
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0]) 
            manufacturer = serie_pt_objet.get_series_details()['series']['Manufacturer'] 
            #si philips on retraitera plus tard 
            if 'philips' in manufacturer.lower() :
                philips_serie.append(serie)
            else : untreated_liste_2.append(serie)

        else : 
            serie_pt_objet = SeriesPT(serie[2]) 
            manufacturer = serie_pt_objet.get_series_details()['series']['Manufacturer'] 
            #si philips on retraitera plus tard 
            if 'philips' in manufacturer.lower() :
                philips_serie.append(serie)
            else : untreated_liste_2.append(serie)

    except Exception as err : 
        print(serie)
        print(err)


print(len(philips_serie))
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_untreated_philips_serie', philips_serie)
print(len(untreated_liste_2))
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_untreated_serie', untreated_liste_2)

203
189


In [3]:
#IMPORT JSON (serie with false mask, except Philips)
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_untreated_serie.json'
untreated_liste_2 = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        untreated_liste_2.append(info)

print(len(untreated_liste_2)) #189

189


In [4]:
#SUR LES LISTES NON PHILIPS, ON REGARDE
#SI ERREUR SUR SUV MAX MEAN SD SEULEMENT SUR DES ELLIPSES, SI OUI => GENERER NIFTI 

erreur_polygone_max = []
erreur_polygone_mean_sd = []
for serie in untreated_liste_2 : 
    print(untreated_liste_2.index(serie))
    try : 
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array

            if 'POLYGON' in mask_objet.ecart_suv_max(nifti_array) :
                print("PROBLEME SUV MAX POLYGON")
                erreur_polygone_max.append(serie)
            
            else : 

                result_mean = mask_objet.ecart_suv_mean(nifti_array)
                result_sd = mask_objet.ecart_SD(nifti_array)
                if (('POLYGON' in result_mean) and ('POLYGON' in result_sd)) : 
                    print("PROBLEME SUV MEAN ET SD POLYGONE")
                    print("ecart mean", result_mean)
                    print("ecart sd", result_sd)
                    erreur_polygone_mean_sd.append(serie)

                else : 
                    #si erreur sur les sd et mean QUE sur les ellipses, on tolère 
                    print("SMALL ERRORS ON ELLIPSE, MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                    #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[2])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                    print("EXPORT NIFTI MASK")


        else : 
            serie_pt_objet = SeriesPT(serie[2]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array

            if 'POLYGON' in mask_objet.ecart_suv_max(nifti_array)  :
                print("PROBLEME SUV MAX POLYGON")
                erreur_polygone_max.append(serie)
            else : 

                result_mean = mask_objet.ecart_suv_mean(nifti_array)
                result_sd = mask_objet.ecart_SD(nifti_array)
                if (('POLYGON' in result_mean) and ('POLYGON' in result_sd)) : 
                    print("PROBLEME SUV MEAN ET SD POLYGONE")
                    print("ecart mean", result_mean)
                    print("ecart sd", result_sd)
                    erreur_polygone_mean_sd.append(serie)

                else : 
                    #si erreur sur les sd et mean QUE sur les ellipses, on tolère 
                    print("SMALL ERRORS ON ELLIPSE, MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                    #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[0])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                    print("EXPORT NIFTI MASK")


    except Exception as err : 
        print(serie)
        print(err)

0
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
1
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
2
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
3
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
4
PROBLEME SUV MAX POLYGON
5
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
6
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
7
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
8
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
9
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
10
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT
EXPORT NIFTI MASK
11
SMALL ERRORS ON ELLIPSE, MASK CORRECT
EXPORT NIFTI PT
EXPORT NIFTI CT

In [5]:
print(len(erreur_polygone_max))
print(len(erreur_polygone_mean_sd))

89
3


In [6]:
#save as json the list of serie which have a mask problem 
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_serie_error_polygone_max', erreur_polygone_max)
write_json_file('/media/salim/DD 2To/AHL2011_NIFTI', 'AHL2011_serie_error_polygone_mean_sd', erreur_polygone_mean_sd)

In [ ]:
#CHECKER LE FLIP : AUCUNE SERIE GENERER AVEC LE FLIP 

In [19]:
#ON FLIP LE MASK ET ON REGARDE SI LES CALCLS SUV MAX MEAN SD SONT BONS
#SI OUI => GENERE NIFTI 

serie_with_flip = []
for serie in serie_error_mask: 
    print(serie_error_mask.index(serie))
    try : 
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array
            if (mask_objet.is_correct_suv(nifti_array, flip = True) == True) : #si mask correct APRES LE FLIP 
                print("MASK CORRECT APRES FLIP")
                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[2])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
                #generation nifti mask
                mask_objet.flip_z(mask_4D)
                mask_4D_flip = mask_objet.mask_array
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D_flip)
                print("EXPORT NIFTI MASK")
                serie_with_flip.append(serie)

        else : 
            serie_pt_objet = SeriesPT(serie[2]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array
            if (mask_objet.is_correct_suv(nifti_array, flip = True) == True) : #si mask correct APRES LE FLIP 
                print("MASK CORRECT APRES FLIP")
                #generation nifti PT
                filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                print("EXPORT NIFTI PT")
                #generation nifti CT
                serie_ct_objet = SeriesCT(serie[0])
                serie_ct_objet.get_numpy_array()
                filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                print("EXPORT NIFTI CT")
                #generation nifti mask
                mask_objet.flip_z(mask_4D)
                mask_4D_flip = mask_objet.mask_array
                filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D_flip)
                print("EXPORT NIFTI MASK")
                serie_with_flip.append(serie)



    except Exception as err : 
        print(serie)
        print(err)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156


In [20]:
#APRES FLIP, REGARDER SI SMALL ERRORS SUR ELLIPSE
#SI PAS DE PROBLEMES SUR LES POLYGONES ON PEUT GENERER LE NIFTI 

In [4]:
#MASK AVEC FLIP FAUX : 
serie_problem = []

for serie in serie_error_mask: 
    print(serie_error_mask.index(serie))
    try : 
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array
            if 'POLYGON' in mask_objet.ecart_suv_max(nifti_array, flip = True) :
                print("PROBLEME SUV MAX POLYGON")
                serie_problem.append(serie)
            else : 
                result_mean = mask_objet.ecart_suv_mean(nifti_array, flip = True)
                result_sd = mask_objet.ecart_SD(nifti_array, flip = True)
                if (('POLYGON' not in result_mean) and ('POLYGON' not in result_sd)) : 
                    #si erreur sur les sd et mean QUE sur les ellipses, on tolère 
                    print("SMALL ERRORS ON ELLIPSE AFTER FLIP, MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                    #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[2])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    mask_objet.flip_z(mask_4D)
                    mask_4D_flip = mask_objet.mask_array
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D_flip)
                    print("EXPORT NIFTI MASK")
                else : #sinon problème sur lles polygones
                    serie_problem.append(serie)





        else : 
            serie_pt_objet = SeriesPT(serie[2]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array
            if 'POLYGON' in mask_objet.ecart_suv_max(nifti_array, flip = True) :
                print("PROBLEME SUV MAX POLYGONE")
                serie_problem.append(serie)
            else : 
                result_mean = mask_objet.ecart_suv_mean(nifti_array, flip = True)
                result_sd = mask_objet.ecart_SD(nifti_array, flip = True)
                if (('POLYGON' not in result_mean) and ('POLYGON' not in result_sd)) : 
                    #si erreur sur les sd et mean QUE sur les ellipses, on tolère 
                    print("SMALL ERRORS ON ELLIPSE AFTER FLIP, MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                    #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[0])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    mask_objet.flip_z(mask_4D)
                    mask_4D_flip = mask_objet.mask_array
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D_flip)
                    print("EXPORT NIFTI MASK")
                else : #sinon problème sur les polygones
                    serie_problem.append(serie)
            

    except Exception as err : 
        print(serie)
        print(err)

0
PROBLEME SUV MAX
1
PROBLEME SUV MAX
2
PROBLEME SUV MAX
3
PROBLEME SUV MAX
4
PROBLEME SUV MAX
5
PROBLEME SUV MAX
6
PROBLEME SUV MAX
7
PROBLEME SUV MAX
8
PROBLEME SUV MAX
9
PROBLEME SUV MAX
10
PROBLEME SUV MAX
11
PROBLEME SUV MAX
12
PROBLEME SUV MAX
13
PROBLEME SUV MAX
14
PROBLEME SUV MAX
15
PROBLEME SUV MAX
16
PROBLEME SUV MAX
17
PROBLEME SUV MAX
18
PROBLEME SUV MAX
19
PROBLEME SUV MAX
20
PROBLEME SUV MAX
21
PROBLEME SUV MAX
22
PROBLEME SUV MAX
23
PROBLEME SUV MAX
24
PROBLEME SUV MAX
25
PROBLEME SUV MAX
26
PROBLEME SUV MAX
27
PROBLEME SUV MAX
28
PROBLEME SUV MAX
29
PROBLEME SUV MAX
30
PROBLEME SUV MAX
31
PROBLEME SUV MAX
32
PROBLEME SUV MAX
33
PROBLEME SUV MAX
34
PROBLEME SUV MAX
35
PROBLEME SUV MAX
36
PROBLEME SUV MAX
37
PROBLEME SUV MAX
38
PROBLEME SUV MAX
39
PROBLEME SUV MAX
40
PROBLEME SUV MAX
41
PROBLEME SUV MAX
42
PROBLEME SUV MAX
43
PROBLEME SUV MAX
44
PROBLEME SUV MAX
45
PROBLEME SUV MAX
46
PROBLEME SUV MAX
47
PROBLEME SUV MAX
48
PROBLEME SUV MAX
49
PROBLEME SUV MAX
50
PROBLEM

In [ ]:
#SERIE WITH ERROR SIZE MATRIX CORONAL 
#ICI FLIP SEMBLE MARCHER 

In [3]:
#ALL SERIE WITH ERROR
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_all_serie_error.json'
error = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        error.append(info)

print(len(error))

9


In [16]:
erreur_suv_max_polyg = []
erreur_suv_mean_sd_polyg = []
treated_serie = []
for serie in error : 
    print(error.index(serie))
    try : 
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array

            if 'POLYGON' in mask_objet.ecart_suv_max(nifti_array) :
                print("PROBLEME SUV MAX POLYGON")
                erreur_suv_max_polyg.append(serie)
                treated_serie.append(serie)
            
            else : 

                result_mean = mask_objet.ecart_suv_mean(nifti_array)
                result_sd = mask_objet.ecart_SD(nifti_array)
                if (('POLYGON' in result_mean) and ('POLYGON' in result_sd)) : 
                    print("PROBLEME SUV MEAN ET SD POLYGONE")
                    print("ecart mean", result_mean)
                    print("ecart sd", result_sd)
                    erreur_suv_mean_sd_polyg.append(serie)

                    treated_serie.append(serie)

                else : 
                    #si erreur sur les sd et mean QUE sur les ellipses, on tolère 
                    print("SMALL ERRORS ON ELLIPSE, MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                    #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[2])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                    print("EXPORT NIFTI MASK")
                    treated_serie.append(serie)


        else : 
            serie_pt_objet = SeriesPT(serie[2]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array

            if 'POLYGON' in mask_objet.ecart_suv_max(nifti_array)  :
                print("PROBLEME SUV MAX POLYGON")
                erreur_suv_max_polyg.append(serie)
                treated_serie.append(serie)
            else : 

                result_mean = mask_objet.ecart_suv_mean(nifti_array)
                result_sd = mask_objet.ecart_SD(nifti_array)
                if (('POLYGON' in result_mean) and ('POLYGON' in result_sd)) : 
                    print("PROBLEME SUV MEAN ET SD POLYGONE")
                    print("ecart mean", result_mean)
                    print("ecart sd", result_sd)
                    erreur_suv_mean_sd_polyg.append(serie)
                    treated_serie.append(serie)

                else : 
                    #si erreur sur les sd et mean QUE sur les ellipses, on tolère 
                    print("SMALL ERRORS ON ELLIPSE, MASK CORRECT")
                    #generation nifti PT
                    filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_pt))
                    print("EXPORT NIFTI PT")
                    #generation nifti CT
                    serie_ct_objet = SeriesCT(serie[0])
                    serie_ct_objet.get_numpy_array()
                    filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
                    serie_ct_objet.export_nifti(os.path.join(nifti_directory, filename_ct))
                    print("EXPORT NIFTI CT")
                    #generation nifti mask
                    filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
                    serie_pt_objet.export_nifti(os.path.join(nifti_directory, filename_mask), mask_4D)
                    print("EXPORT NIFTI MASK")
                    treated_serie.append(serie)


    except Exception as err : 
        print(serie)
        print(err)

0
['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101031030/1.2.840.113704.1.111.3556.1391590354.9/1.2.840.113704.7.32.0.28.2.10.48.18673.58372.2.2100.0.1391592740/1.2.840.113704.7.32.0.28.2.10.48.18673.58372.2.2100.0.1391592740', 'PT', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101031030/1.2.840.113704.1.111.3556.1391590354.9/1.2.840.113704.7.32.1.2.840.113704.1.111.2676.1391590761.8/1.2.840.113704.7.32.1.2.840.113704.1.111.2676.1391590761.8', 'CT', 'gda_in_feb 5_2014.csv']
index 255 is out of bounds for axis 0 with size 255
1
['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101021024/1.2.250.1.59.470.940100027.1.20140110123209.2625.18891/1.2.840.113704.1.111.5356.1389356368.8/1.2.840.113704.1.111.5356.1389356368.8', 'CT', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101021024/1.2.250.1.59.470.940100027.1.20140110123209.2625.18891/1.3.46.670589.28.2.10.48.18554.65512.2.408.0.1389358191/1.3.46.670589.28.2.10.48.18554.65512.2.408.0.1389358191', 'PT', 'dja_fa_jan 10_

In [4]:
test = error[0]
print(test)
error.remove(test)

['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101031030/1.2.840.113704.1.111.3556.1391590354.9/1.2.840.113704.7.32.0.28.2.10.48.18673.58372.2.2100.0.1391592740/1.2.840.113704.7.32.0.28.2.10.48.18673.58372.2.2100.0.1391592740', 'PT', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101031030/1.2.840.113704.1.111.3556.1391590354.9/1.2.840.113704.7.32.1.2.840.113704.1.111.2676.1391590761.8/1.2.840.113704.7.32.1.2.840.113704.1.111.2676.1391590761.8', 'CT', 'gda_in_feb 5_2014.csv']


In [5]:
print(len(error))

8


In [23]:
test = error[1]
print(test)

['/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101051008/1.2.250.1.74.20120622095500.60000021132086/1.3.46.670589.28.2.7.2202872868.2.24584.0.1340366188/1.3.46.670589.28.2.7.2202872868.2.24584.0.1340366188', 'PT', '/media/salim/DD 2To/AHL2011_Validated_DICOMS/13011101051008/1.2.250.1.74.20120622095500.60000021132086/1.2.840.113704.1.111.6284.1340361537.8/1.2.840.113704.1.111.6284.1340361537.8', 'CT', 'bor_fr_jun 22_2012.csv']


In [27]:
false_nifti_directory = '/media/salim/DD 2To/AHL2011_FALSE_MASK_NIFTI'
serie_pt_objet = SeriesPT(test[0])
nifti_array = serie_pt_objet.get_numpy_array()
study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
size_matrix = serie_pt_objet.get_size_matrix()

filename_pt = study_uid+'_'+'nifti_'+'PT'+'.nii'
serie_pt_objet.export_nifti(os.path.join(false_nifti_directory, filename_pt))

serie_ct_objet = SeriesCT(test[2])
filename_ct = study_uid+'_'+'nifti_'+'CT'+'.nii'
serie_ct_objet.export_nifti(os.path.join(false_nifti_directory, filename_ct))



print(size_matrix)
mask_objet = MaskBuilder(os.path.join(csv_directory, test[4]), size_matrix)
mask_4D = mask_objet.mask_array

mask_objet.flip_z(mask_4D)
mask_4D_flip = mask_objet.mask_array

filename_mask = study_uid+'_'+'nifti_'+'mask'+'.nii'
serie_pt_objet.export_nifti(os.path.join(false_nifti_directory, filename_mask), mask_4D_flip)





[144, 144, 234]
1
shape_matrice a remplir (234, 144, 144)
shape après coronal to axial (144, 144, 234)
shape_ 3D array après get mask (144, 144, 234)
(144, 144, 234, 30)
coronal
2
shape_matrice a remplir (234, 144, 144)
shape après coronal to axial (144, 144, 234)
shape_ 3D array après get mask (144, 144, 234)
(144, 144, 234, 30)
coronal
3
shape_matrice a remplir (234, 144, 144)
shape après coronal to axial (144, 144, 234)
shape_ 3D array après get mask (144, 144, 234)
(144, 144, 234, 30)
coronal
4
shape_matrice a remplir (234, 144, 144)
shape après coronal to axial (144, 144, 234)
shape_ 3D array après get mask (144, 144, 234)
(144, 144, 234, 30)
coronal
5
shape_matrice a remplir (234, 144, 144)
shape après coronal to axial (144, 144, 234)
shape_ 3D array après get mask (144, 144, 234)
(144, 144, 234, 30)
coronal
6
shape_matrice a remplir (234, 144, 144)
shape après coronal to axial (144, 144, 234)
shape_ 3D array après get mask (144, 144, 234)
(144, 144, 234, 30)
coronal
7
shape_matr

In [26]:
mask_objet.calcul_suv(nifti_array, flip = True)

ROI : 1
ROI : 2
ROI : 3
ROI : 4
ROI : 5
ROI : 6
ROI : 7
ROI : 8
ROI : 9
ROI : 10
ROI : 11
ROI : 12
ROI : 13
ROI : 14
ROI : 15
ROI : 16
ROI : 17
ROI : 18
ROI : 19
ROI : 20
ROI : 21
ROI : 22
ROI : 23
ROI : 24
ROI : 25
ROI : 26
ROI : 27
ROI : 28
ROI : 29
ROI : 30


{1: {'SUV_max': 20.72, 'SUV_mean': 11.18, 'SD': 2.39},
 2: {'SUV_max': 14.6, 'SUV_mean': 7.88, 'SD': 1.67},
 3: {'SUV_max': 17.41, 'SUV_mean': 10.26, 'SD': 2.33},
 4: {'SUV_max': 15.43, 'SUV_mean': 8.95, 'SD': 2.05},
 5: {'SUV_max': 15.55, 'SUV_mean': 9.07, 'SD': 2.22},
 6: {'SUV_max': 12.09, 'SUV_mean': 6.88, 'SD': 1.48},
 7: {'SUV_max': 9.39, 'SUV_mean': 5.46, 'SD': 1.38},
 8: {'SUV_max': 16.78, 'SUV_mean': 9.03, 'SD': 1.99},
 9: {'SUV_max': 14.28, 'SUV_mean': 7.79, 'SD': 1.68},
 10: {'SUV_max': 10.44, 'SUV_mean': 5.64, 'SD': 1.39},
 11: {'SUV_max': 13.41, 'SUV_mean': 7.04, 'SD': 1.22},
 12: {'SUV_max': 13.41, 'SUV_mean': 7.26, 'SD': 1.5},
 13: {'SUV_max': 8.69, 'SUV_mean': 5.1, 'SD': 1.24},
 14: {'SUV_max': 6.05, 'SUV_mean': 3.13, 'SD': 0.67},
 15: {'SUV_max': 7.46, 'SUV_mean': 3.87, 'SD': 0.92},
 16: {'SUV_max': 8.34, 'SUV_mean': 4.87, 'SD': 1.12},
 17: {'SUV_max': 10.22, 'SUV_mean': 5.66, 'SD': 1.37},
 18: {'SUV_max': 15.21, 'SUV_mean': 7.95, 'SD': 1.62},
 19: {'SUV_max': 9.19, 'S

In [3]:
#SERIE WITH ERROR SUV MAX POLYGONE
json_path = '/media/salim/DD 2To/AHL2011_NIFTI/AHL2011_error_suv_max_polygone.json'
error_suv = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        error_suv.append(info)

print(len(error_suv))

149


In [4]:
doublon = error_suv[91]
error_suv.remove(doublon)

In [5]:
len(error_suv)

148

In [ ]:
#RAJOUTER METHODE POUR SEUIL DES MASKS
#1 RECUPERER LIST POINTS APRES LE MASK (METTRE EN PARAMETRE LA LISTE DES POINTS DANS MA FONCTION)
# ET RECUPERER LE SEUIL
# 2 REGARDER DANS LE NIFTI A CES POINTS SI LA VALEUR DU SUV DEPASSE LE SEUIL*NP MAX OU SEUIL
# 3 SI VALEUR PLUS PETITE QUE LE SEUIL, ON MET 0 DANS LE MASK AUX COORDONNEES DE LA ROI
#RETURN 4D MASK SEUILLE

In [6]:
#GENERATE PDF WITH MIP ANGLE 0 OF PET AND MASK FOR EVERY SERIE

from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *
mip_directory = '/media/salim/DD 2To/AHL2011_MIP'



path_mip = []
for serie in error_suv : 
    index = error_suv.index(serie)
    print(index)
    subliste = []
    if serie[1] =='PT' :
        serie_pt_objet = SeriesPT(serie[0])
        study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
        nifti_array = serie_pt_objet.get_numpy_array()
        angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 3.0)             #mip.png PET
        subliste.append(angle_filename)

        size_matrix = serie_pt_objet.get_size_matrix()
        mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
        mask_4D = mask_objet.mask_array
        number_roi = mask_4D.shape[3]
        threshold = mask_objet.details_rois['SUVlo']
        #seuillage
        mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array,  flip = False)
        angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, borne_max = 5.0)
        #mip.png MASK
        subliste.append(angle_filename_mask)

        path_mip.append(subliste)

        




    else : 
        serie_pt_objet = SeriesPT(serie[2])
        study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
        nifti_array = serie_pt_objet.get_numpy_array()
        angle_filename = mip_projection(nifti_array, 0, mip_directory, study_uid, borne_max = 3.0) 
        #mip.png PET
        subliste.append(angle_filename)
        size_matrix = serie_pt_objet.get_size_matrix()
        mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
        mask_4D = mask_objet.mask_array
        number_roi = mask_4D.shape[3]
        threshold = mask_objet.details_rois['SUVlo']
        #seuillage
        mask_4D = threshold_mask(mask_4D, mask_objet.details_rois, nifti_array, flip = False)
        angle_filename_mask = mip_projection_4D(mask_4D, 0, mip_directory, study_uid, number_roi, borne_max = 5.0)
        #mip.png MASK
        subliste.append(angle_filename_mask)

        path_mip.append(subliste)



0
taille mask :  (200, 200, 481)


IndexError: too many indices for array

In [ ]:
print(len(path_mip))

In [ ]:
#create pdf 
filename = mip_directory+ '.pdf'
create_pdf_mip(path_mip, filename )

In [20]:
import os.path, time 


result = {}
for serie in error_suv :
    print(error_suv.index(serie))
    subliste = {}
    if serie[1] =='PT' :
        serie_pt_objet = SeriesPT(serie[0])
        study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
        subliste['StudyUID'] = study_uid
        subliste['Manufacturer'] = serie_pt_objet.get_series_details()['series']['Manufacturer']
        subliste['Units'] = serie_pt_objet.get_series_details()['series']['Units']
        nifti_array = serie_pt_objet.get_numpy_array()
        size_matrix = serie_pt_objet.get_size_matrix()

        mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
        number_roi = mask_objet.mask_array.shape[3]
        axis = []
        for roi in range(1, number_roi+1):
            axis.append(mask_objet.details_rois[roi]['type_number'])
        subliste['Axis'] = axis
        subliste['Last_Modif'] = time.ctime(os.path.getmtime(os.path.join(csv_directory, serie[4])))

        result[error_suv.index(serie)] = subliste

        print(subliste)



    else : 
        serie_pt_objet = SeriesPT(serie[2])
        study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
        subliste['StudyUID'] = study_uid
        subliste['Manufacturer'] = serie_pt_objet.get_series_details()['series']['Manufacturer']
        subliste['Units'] = serie_pt_objet.get_series_details()['series']['Units']
        nifti_array = serie_pt_objet.get_numpy_array()
        size_matrix = serie_pt_objet.get_size_matrix()


        mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
        number_roi = mask_objet.mask_array.shape[3]
        axis = []
        for roi in range(1, number_roi+1):
            axis.append(mask_objet.details_rois[roi]['type_number'])
        subliste['Axis'] = axis
        subliste['Last_Modif'] = time.ctime(os.path.getmtime(os.path.join(csv_directory, serie[4])))

        result[error_suv.index(serie)] = subliste

        print(subliste)

facturer': 'GE MEDICAL SYSTEMS', 'Units': 'BQML', 'Axis': [1, 1, 1, 1, 1, 1, 1, 11, 11, 11, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 11, 1, 1, 1, 1, 1, 1], 'Last_Modif': 'Sun Sep 20 19:18:09 2015'}
68
{'StudyUID': '1.3.51.0.1.1.10.49.10.222.987570.985613', 'Manufacturer': 'GE MEDICAL SYSTEMS', 'Units': 'BQML', 'Axis': [1, 1, 1, 11, 11, 1, 11, 1, 11, 11, 1, 11, 11, 1, 11, 1, 11, 1, 1, 11, 11, 11, 11, 1, 11, 11, 11, 1], 'Last_Modif': 'Fri Oct  2 16:53:44 2015'}
69
{'StudyUID': '2.16.56.465769650.1.57524848525053565249', 'Manufacturer': 'CPS', 'Units': 'BQML', 'Axis': [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 1, 1, 1, 1], 'Last_Modif': 'Fri Dec  4 10:18:28 2015'}
70
{'StudyUID': '2.16.56.465769650.2551326252.2798663604', 'Manufacturer': 'SIEMENS', 'Units': 'BQML', 'Axis': [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 

In [7]:
all_false_roi = []
false_roi = []
for serie in error_suv : 
    print(error_suv.index(serie))
    try : 
        if serie[1] == 'PT' : 
            serie_pt_objet = SeriesPT(serie[0]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array
            number_roi = mask_4D.shape[3]
            ecart_suv_max = mask_objet.ecart_suv_max(nifti_array)
            number_false_roi = len(ecart_suv_max)/3
            if number_false_roi == number_roi : 
                print("TOUTE LES ROIS FAUSSES")
                all_false_roi.append(serie)
                ecart_flip = mask_objet.ecart_suv_max(nifti_array, flip = True)
                number_false_roi_flip = len(ecart_flip)/3
                print("NOMBRE DE ROI ", number_roi)
                print("ROI FAUSSE =", number_false_roi)


            else :
                print("CERTAINE ROIS FAUSSE")
                liste = ecart_suv_max[::3]
                print("NBR :", int(number_false_roi)) 
                print("LISTE ROI FAUSSE :", liste)
                false_roi.append(serie)
                false_roi.append(int(number_false_roi))
                false_roi.append(liste)
            




        else : 
            serie_pt_objet = SeriesPT(serie[2]) 
            nifti_array = serie_pt_objet.get_numpy_array()
            study_uid = serie_pt_objet.get_series_details()['study']['StudyInstanceUID']
            size_matrix = serie_pt_objet.get_size_matrix()
            mask_objet = MaskBuilder(os.path.join(csv_directory, serie[4]), size_matrix)
            mask_4D = mask_objet.mask_array
            number_roi = mask_4D.shape[3]
            ecart_suv_max = mask_objet.ecart_suv_max(nifti_array)
            number_false_roi = len(ecart_suv_max)/3
            if number_false_roi == number_roi : 
                print("TOUTE LES ROIS FAUSSES")
                all_false_roi.append(serie)
                all_false_roi.append(serie)
                ecart_flip = mask_objet.ecart_suv_max(nifti_array, flip = True)
                number_false_roi_flip = len(ecart_flip)/3
                print("NOMBRE DE ROI ", number_roi)
                print("ROI FAUSSE =", number_false_roi)


            else :
                print("CERTAINE ROIS FAUSSE")
                liste = ecart_suv_max[::3]
                print("NBR :", int(number_false_roi)) 
                print("LISTE ROI FAUSSE :", liste)
                false_roi.append(serie)
                false_roi.append(int(number_false_roi))
                false_roi.append(liste)


    except Exception as err : 
        print(serie)
        print(err)

0
CERTAINE ROIS FAUSSE
NBR : 2
LISTE ROI FAUSSE : [20, 41]
1
TOUTE LES ROIS FAUSSES
NOMBRE DE ROI  10
ROI FAUSSE = 10.0
2
TOUTE LES ROIS FAUSSES
NOMBRE DE ROI  16
ROI FAUSSE = 16.0
3
TOUTE LES ROIS FAUSSES
NOMBRE DE ROI  5
ROI FAUSSE = 5.0
4
TOUTE LES ROIS FAUSSES
NOMBRE DE ROI  30
ROI FAUSSE = 30.0
5
TOUTE LES ROIS FAUSSES
NOMBRE DE ROI  20
ROI FAUSSE = 20.0
6
CERTAINE ROIS FAUSSE
NBR : 1
LISTE ROI FAUSSE : [17]
7
CERTAINE ROIS FAUSSE
NBR : 2
LISTE ROI FAUSSE : [18, 27]
8
CERTAINE ROIS FAUSSE
NBR : 2
LISTE ROI FAUSSE : [9, 40]
9
CERTAINE ROIS FAUSSE
NBR : 1
LISTE ROI FAUSSE : [8]
10
TOUTE LES ROIS FAUSSES
NOMBRE DE ROI  46
ROI FAUSSE = 46.0
11
TOUTE LES ROIS FAUSSES
NOMBRE DE ROI  15
ROI FAUSSE = 15.0
12
CERTAINE ROIS FAUSSE
NBR : 69
LISTE ROI FAUSSE : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 

KeyboardInterrupt: 